In [2]:
# import necessary library
import pandas as pd

In [3]:
# import the data 
data = pd.read_excel('data_goodreads.xlsx')
data.head()

,user_id,gender,ratings_count,reviews_count,year_joined,young_age,location,friends,fav_genres,groups_count,2018,avg_page_18,2019,avg_page_19,2020,avg_page_20
0,1,F,806,34,2014,1,Asia,103,"Chick-lit, Christian, Ebooks, Fantasy, Fiction...",6,0,209,0,199,1,297
1,2,F,822,90,2013,0,USA,225,"Chick-lit, Contemporary, Fantasy, Fiction, His...",0,1,388,1,383,1,355
2,3,F,114,4,2014,1,Asia,7,"Chick-lit, Contemporary, Ebooks, Romance, Self...",1,1,288,1,292,0,284
3,4,F,1265,630,2015,1,Asia,588,"Fantasy, Historical fiction, Manga, and Young-...",1,1,302,1,346,1,380
4,5,F,380,1,2014,0,EU,56,"Comics, Fantasy, Historical fiction, Humor and...",1,1,327,1,628,0,278


In [4]:
# clean the fav_genres column 
for row in range(len(data['fav_genres'])):
    data['fav_genres'][row] = data['fav_genres'][row].replace('\xa0','').replace(' and','').split(',')

C:\Users\glady\AppData\Local\Temp/ipykernel_1516/468483999.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['fav_genres'][row] = data['fav_genres'][row].replace('\xa0','').replace(' and','').split(',')


In [5]:
# function to clean the genres fiction
# as there are many types of fiction, it's better to put it just as fiction 
def clean_fiction(li):
    new = []
    for text in li:
        l = text.split(' ')
        if len(l) > 1:
            if l[1].lower() == 'fiction':
                new.append('Fiction')
            else:
                new.append(' '.join(l).strip())
        else:
            new.append(' '.join(l).strip())

    return new

In [6]:
# apply the above function and make a new column of it 
data['fav_genres2'] = data['fav_genres'].apply(clean_fiction)

In [7]:
# clean the genres and to make the population smaller, so we can categorize preferences later
def clean_genres(li):
    new = []
    for l in li:
        if l.lower() == 'ebooks' or l.lower() == 'cookbooks' or l.lower() == 'book club' or l.lower() == 'poetry' or l.lower() == 'sports' or l.lower() == 'art' or l.lower() == 'lgbt' or l.lower() == 'gay lesbian' or l.lower() == 'new-adult' or l.lower() == 'disability' or l.lower() == 'short-stories' or l.lower() == 'feminism' or l.lower() == 'historical-mystery':
            pass
        elif l.lower() == 'suspense' or l.lower() == 'crime' or l.lower() == 'mystery':
            new.append('Thriller')
        elif l.lower() == 'paranormal':
            new.append('Horror')
        elif l.lower() == 'personal-development':
            new.append('Self help')
        elif l.lower() == 'chick-lit':
            new.append('Romance')
        elif l.lower() == 'contemporary' or l.lower() == 'fantasy':
            new.append('Fiction')
        elif l.lower() == 'manga' or l.lower() == 'graphic novels':
            new.append('Comics')
        elif l.lower() == 'memoir':
            new.append('Biography')
        elif l.lower() == 'leadership':
            new.append('Business')
        elif l.lower() == 'christian' or l.lower() == 'spirituality':
            new.append('Religion')
        elif l.lower() == 'music':
            new.append('Art')
        else:
            new.append(l)

    return new

In [8]:
# apply above function 
data['fav_genres2'] = data['fav_genres2'].apply(clean_genres)

In [9]:
# categorize genres whether it is fiction or nonfiction  
def categorize(li):
    new = []
    for l in li:
        if l.lower() == 'thriller' or l.lower() == 'childrens' or l.lower() == 'horror' or l.lower() == 'romance' or l.lower() == 'comics' or l.lower() == 'religion' or l.lower() == 'young-adult' or l.lower() == 'humor comedy' or l.lower() == 'classics' or l.lower() == "children's" or l.lower() == 'young adult' or l.lower() == 'epic-fantasy' or l.lower() == 'dark-fantasy' or l.lower() == 'mythology':
            new.append('Fiction')
        elif l.lower() == 'self help' or l.lower() == 'biography' or l.lower() == 'business' or l.lower() == 'psychology' or l.lower() == 'science' or l.lower() == 'history' or l.lower() == 'philosophy' or l.lower() == 'travel' or l.lower() == 'politics' or l.lower() == 'nonfiction' or l.lower() == 'historical' or l.lower() == 'war':
            new.append('Non-fiction')
        else:
            new.append(l)
    
    return new 

In [10]:
# apply above function and make a new column
data['category'] = data['fav_genres2'].apply(categorize)

In [11]:
# remove all other genres that is not included in fiction or non-fiction
# as there are many niche genres 
def fic_nonfic(li):
    new = []
    for l in li:
        if l.lower() == 'fiction' or l.lower() == 'non-fiction':
            new.append(l)
        else:
            pass
        
    return new

In [12]:
# apply above function
data['category'] = data['category'].apply(fic_nonfic)

In [13]:
# copy the column 'category'
data['category_simplified'] = data['category'].copy()

In [14]:
# make the 'category' column into a set so we only have either fiction, non-fiction, or both
data['category'] = data['category'].apply(set)

In [15]:
# function to make a boolean value whether people prefer fiction (1) or nonfiction (0)
# based on whether they prefer more fiction genres or non-fiction genres
def simplifying(li):
    for row in li:
        if row.count('Fiction') > row.count('Non-fiction'):
            return('1')
        else:
            return('0')
    

In [16]:
# apply above function 
data['category_simplified'] = data['category_simplified'].apply(simplifying)

In [17]:
# function to make the 'gender' column into boolean value
def gender_category(li):
    for row in li:
        if row == 'F':
            return('1')
        else:
            return('0')

In [18]:
# apply above function
data['gender'] = data['gender'].apply(gender_category)

In [19]:
# the value of people who prefer fiction and non-fiction
data['category_simplified'].value_counts()

1    66
0    34
Name: category_simplified, dtype: int64

In [20]:
# function to sum the total amount of challenge completed(0-3)
def total_page (row):
    x = row['avg_page_18'] + row['avg_page_19'] + row['avg_page_20']
    return x

# function to sum the total amount of avg page from 2018 to 2020
def challenge_completed (row):
    x = row[2018] + row[2019] + row[2020]
    return x

In [21]:
# apply above functions and make a new column 
data['total_page'] = data.apply(total_page, axis=1)
data['challenge_completed'] = data.apply(challenge_completed, axis=1)
data

,user_id,gender,ratings_count,reviews_count,year_joined,young_age,location,friends,fav_genres,groups_count,...,avg_page_18,2019,avg_page_19,2020,avg_page_20,fav_genres2,category,category_simplified,total_page,challenge_completed
0,1,1,806,34,2014,1,Asia,103,"[Chick-lit, Christian, Ebooks, Fantasy, Fictio...",6,...,209,0,199,1,297,"[Romance, Religion, Fiction, Fiction, Romance,...","{Non-fiction, Fiction}",1,705,1
1,2,1,822,90,2013,0,USA,225,"[Chick-lit, Contemporary, Fantasy, Fiction, Hi...",0,...,388,1,383,1,355,"[Romance, Fiction, Fiction, Fiction, Fiction, ...","{Non-fiction, Fiction}",1,1126,3
2,3,1,114,4,2014,1,Asia,7,"[Chick-lit, Contemporary, Ebooks, Romance, Sel...",1,...,288,1,292,0,284,"[Romance, Fiction, Romance, Self help, Young-a...","{Non-fiction, Fiction}",1,864,2
3,4,1,1265,630,2015,1,Asia,588,"[Fantasy, Historical fiction, Manga, Young-adult]",1,...,302,1,346,1,380,"[Fiction, Fiction, Comics, Young-adult]",{Fiction},1,1028,3
4,5,1,380,1,2014,0,EU,56,"[Comics, Fantasy, Historical fiction, Humor Co...",1,...,327,1,628,0,278,"[Comics, Fiction, Fiction, Humor Comedy, Psych...","{Non-fiction, Fiction}",1,1233,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,0,573,413,2012,1,Asia,408,"[Adult Fiction, Book Club, Comics, Fantasy, Fi...",5,...,381,0,263,0,319,"[Fiction, Comics, Fiction, Fiction, Fiction, H...",{Fiction},1,963,0
96,97,0,202,30,2012,1,Asia,364,"[Classics, Fiction, Memoir, Philosophy, Poetry...",3,...,271,1,280,1,274,"[Classics, Fiction, Biography, Philosophy, Psy...","{Non-fiction, Fiction}",1,825,3
97,98,0,36,0,2018,1,Asia,17,"[Classics, History, Non-fiction, Philosophy, P...",0,...,303,0,298,0,328,"[Classics, History, Non-fiction, Philosophy, P...","{Non-fiction, Fiction}",1,929,0
98,99,0,1121,1034,2012,0,Asia,537,"[Fantasy, Fiction, Romance, Young-adult, War]",6,...,317,1,269,1,250,"[Fiction, Fiction, Romance, Young-adult, War]","{Non-fiction, Fiction}",1,836,3


In [22]:
# create a csv 
data.to_csv('goodreads_clean.csv')